In [2]:
import requests

requests.get(url='http://asdfasdfiuyhkjjkhjkhlkjasduihkjlkjahsdlkjf.edu')

ConnectionError: HTTPConnectionPool(host='asdfasdfiuyhkjjkhjkhlkjasduihkjlkjahsdlkjf.edu', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff4736fa0f0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
def parse_one_gpu(gpu_id, region='us', get_memory_type=False):
    prefix = '' if region == 'us' else '{0}.'.format(region)
    url = 'http://{0}pcpartpicker.com/product/{1}'.format(prefix, gpu_id)
    params = {
        'history_days': 730
    }
    scrapped = False
    amount = 1
    while not scrapped:
        try:
            connected = False
            tries = 10
            while not connected:
                try:
                    sleep(0.5)
                    resp = requests.get(url=url, params=params)
                    connected = True
                except:
                    print('connection problem, trying again...')
                    tries -= 1
                    if tries < 0:
                        raise
            soup = BeautifulSoup(resp.text)
            # find memory type
            actual_idx = -1
            mem_type = None
            if get_memory_type:
                specs = soup.find('div', {'class': 'specs block'})
                for i, line in enumerate(specs):
                    if i == actual_idx:
                        mem_type = line.strip()
                        break
                    elif line.string == 'Memory Type':
                        actual_idx = i+1
            # find price data
            raw_data = []
            parsed_data = []
            scripts = soup.findAll('script')
            for script in scripts:
                if 'phistmulti' in script.text:
                    data = script.prettify().split('\n')
                    for line in data:
                        if 'phistmulti' in line:
                            idx = line.index('[')
                            price_hist = line[idx:-1]
                            raw_data = loads(price_hist)
                            break
            for mer_row in raw_data:
                merchant = mer_row['label'].replace('&amp;', ' ')
                for value in mer_row['data']:
                    price = value[1]
                    if price is None:
                        continue
                    price = float(price) / 100
                    utc_time = datetime.utcfromtimestamp(value[0]/1000)
                    temp_dict = {
                        'prodId': gpu_id,
                        'date': utc_time.strftime("%Y-%m-%d"),
                        'price': price,
                        'region': region,
                        'merchant': merchant
                    }
                    parsed_data.append(temp_dict)
            scrapped = True
        except:
            print('CAPTCHA problem, trying again... {0}'.format(amount))
            amount += 1
            if amount > 100:
                raise
            sleep(5)
    return parsed_data, mem_type